<a href="https://colab.research.google.com/github/marcocucei/TI_I_Practica4/blob/main/TI_I_Practica4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 4 - Dashboard Random Walk + Path Length/MSD

**Nombre:** Marco Antonio Hernández Preciado  
**e-mail:** marco.hpreciado@alumnos.udg.mx